# Búsqueda semántica con nubes de puntos
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \frac{1}{2|U|} \sum_{u \in U} \min_{x \in V} d(u, x) + \frac{1}{2|V|}\sum_{v \in V} \min_{x \in U} d(v, x) $$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{tp}$

$$ H^\textsf{pt}_d(U, V) = \max \left\{
        \max_{u \in U} \left\{\textsf{pt}(u, U) \min_{x \in V} d(u, x)\right\},
        \max_{v \in V} \left\{\textsf{pt}(v, V) \min_{x \in U} d(v, x)\right\}
     \right\} $$

# Notas adicionales
La versión de $H^\textsf{pt}_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")

using SimilaritySearch, TextSearch, Plots, Word2Vec, KNearestCenters, LinearAlgebra, HypertextLiteral, JSON, CodecZlib, Random
using Embeddings: load_embeddings, FastText_Text
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
function word2vec_embeddings(textconfig, corpus, embfile; dim=32, w2vtmp=tempfile(pwd()))
    isfile(embfile) && return
    tcorpus = tokenize_corpus(textconfig, corpus)
    voc = Vocabulary(textconfig, tcorpus)
    fvoc = filter_tokens(voc) do t
        5 <= t.ndocs <= 2000
    end
    valid = Set(fvoc.token)

    open(w2vtmp, "w") do f
        for tokens in tcorpus
            tokens = filter!(t -> t in valid, tokens)
            if length(tokens) > 7 # minimum number of tokens in a message
                println(f, join(tokens, ' '))
            end
        end
    end

    word2vec(w2vtmp, embfile, size=dim, iter=15, threads=Threads.nthreads()-1)
end


word2vec_embeddings (generic function with 1 method)

# Cargando datos y creando word embeddings a la medida

In [3]:
textconfig = TextConfig(; nlist=[1], qlist=[], slist=[], lc=true,
    group_usr=true, group_url=true, group_num=true, del_diac=true, del_punc=true)

display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_news()
embfile = "../data/news.w2v.vec"
# isfile(embfile) && rm(embfile)
word2vec_embeddings(textconfig, D.corpus, embfile; dim=64, w2vtmp=joinpath(pwd(), "w2v.tmp"))
E = load_embeddings(FastText_Text, embfile)

@show unique(D.labels)

<h1>Cargando el corpus</h1>

unique(D.labels) = ["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau", "AristeguiOnline", "el_pais", "EFEnoticias", "PublimetroMX", "PublimetroChile", "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"]


25-element Vector{String}:
 "AdriDelgadoRuiz"
 "El_Universal_Mx"
 "CNNEE"
 "NTN24"
 "UniNoticias"
 "TelemundoNews"
 "SinEmbargoMX"
 "Reforma"
 "abc_es"
 "azucenau"
 "AristeguiOnline"
 "el_pais"
 "EFEnoticias"
 "PublimetroMX"
 "PublimetroChile"
 "Radio_Formula"
 "RicardoAlemanMx"
 "epigmenioibarra"
 "Milenio"
 "LaRazon_mx"
 "abrahamendieta"
 "PublimetroCol"
 "teleSURtv"
 "bbcmundo"
 "julioastillero"

In [4]:
T = let
    voc = Vocabulary(textconfig, D.corpus; thesaurus=E.vocab)
    model = VectorModel(BinaryGlobalWeighting(), TpWeighting(), voc)
    (; textconfig, model, voc)
end

(textconfig = TextConfig(true, false, true, true, true, true, false, true, Int8[], Int8[1], Skipgram[], IdentityTokenTransformation()), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=9599, train-n=30244, maxoccs=1929}, voc = Vocabulary(["</s>", "este", "esta", "ha", "anos", "sobre", "sus", "contra", "mexico", "presidente"  …  "recuperarse", "sso", "peter", "louis", "cruces", "nolleva", "acaboelpaseviernes", "salvadorena", "frases", "mayorkas"], Int32[0, 1929, 1929, 1523, 1504, 1452, 1447, 1420, 1347, 1245  …  7, 5, 7, 6, 6, 5, 5, 5, 6, 5], Int32[0, 1929, 1929, 1523, 1504, 1452, 1447, 1420, 1347, 1245  …  7, 5, 7, 6, 6, 5, 5, 5, 6, 5], Dict{String, UInt32}("focos" => 0x00002213, "corresponden" => 0x0000175b, "nocturnos" => 0x000022ba, "chocolate" => 0x00002041, "#videos" => 0x0000177a, "viajara" => 0x0000244c, "inutil" => 0x00001978, "exhibir" => 0x00001ff3, "depositos" => 0x00001eb4, "libertad" => 0x00000210…), 30244))

In [5]:
cloud(text, E, T) = vectorize(T.model, T.textconfig, text; normalize=false)
cloud_corpus(corpus, E, T) = vectorize_corpus(T.model, T.textconfig, corpus; normalize=false)

cloud_corpus (generic function with 1 method)

In [6]:
struct AWMD <: SemiMetric
    X::MatrixDatabase
    results::Vector{KnnResult}
    dcos::NormalizedCosineDistance
end

AWMD(X) = AWMD(X, [KnnResult(1) for _ in 1:Threads.nthreads()], NormalizedCosineDistance())

const PI_2 = convert(Float32, pi / 2)


@inline function nn_(X, res, dcos, u, V)
    res = reuse!(res)

    for (vid, _) in V
        push!(res, vid, evaluate(dcos, u, X[vid]))
    end
    
    minimum(res)
end

function onesidewmd(X, res, dcos, U, V)
    d::Float32 = zero(Float32)
    @fastmath for (uid, w) in U
        if !haskey(V, uid) # evaluates to zero
            d = max(d, w * nn_(X, res, dcos, X[uid], V))
        end
    end
    
    d
end

function SimilaritySearch.evaluate(wmd::AWMD, U::T, V::T) where T
    length(U) == 1 && haskey(U, 0) && return PI_2
    length(V) == 1 && haskey(V, 0) && return PI_2
    res = wmd.results[Threads.threadid()]
    dcos = wmd.dcos

    max(onesidewmd(wmd.X, res, dcos, U, V), onesidewmd(wmd.X, res, dcos, V, U))
end


In [7]:
db = VectorDatabase(cloud_corpus(D.corpus, E, T))
for c in eachcol(E.embeddings)
    normalize!(c)
end

dist = AWMD(MatrixDatabase(E.embeddings))

AWMD(MatrixDatabase{Matrix{Float32}}(Float32[0.17953031 -0.12504481 … 0.032049984 0.049792238; 0.19821823 0.037016124 … -0.014358723 0.19184345; … ; -0.09407112 0.030588793 … 0.109720536 0.041526493; 0.09679824 -0.0152774 … 0.15423927 0.029537214]), KnnResult[KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)  …  KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1), KnnResult(Int32[], Float32[], 1)], NormalizedCosineDistance())

In [8]:
#=recall = 0.8
index = SearchGraph(; dist, db, verbose=true)
#index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
index!(index)
optimize!(index, MinRecall(recall))
=#

In [9]:
index = ParallelExhaustiveSearch(; db, dist);

In [10]:
Threads.nthreads()

64

In [11]:
#@time res, _ = search(ex, db[111], KnnResult(10))
#=@time res, _ = search(ex, cloud("covid en México", E, T), KnnResult(10))

for (id_, dist_) in res
    display(@htl """<div style="padding: 0.5em;">
        <span style="width: 25%; margin: 0.25em;"> $(id_ => round(dist_; digits=3))</span>
        <span style="background-color: rgb(120, 60, 60); margin: 0.2em;">$(D.text[id_])</span>
    </div>""")
end
=#

### Búsqueda y presentación de los resultados

In [12]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = cloud(qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados para "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [13]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "calentamiento global", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  1.367959 seconds (3.04 M allocations: 237.441 MiB, 23.55% gc time, 99.51% compilation time)


1,19737,0.067,azucenau,"La jefa de gobierno de la Ciudad de México, Claudia Sheinbaum, aseguró que ella y el presidente de la República, Andrés Manuel López Obrador, “somos parte del mismo movimiento”, y que jamás será líder de oposición, pues “somos parte de la Cuarta Transformación”. https://t.co/FYoh0GuROr"
2,9748,0.11,SinEmbargoMX,"🚨 #URGENTE | El Gobierno de EU ofrece 5 millones de dólares por información que conduzca al arresto y condena de los mexicanos Ovidio Guzmán López, Iván Archivaldo Guzmán Salazar, Jesús Alfredo Guzmán Salazar y Joaquín Guzmán López, hijos de “El Chapo” https://t.co/S8eyJEDNTX https://t.co/YSXPkhUM7k"
3,27745,0.116,Reforma,"El Presidente Andrés Manuel López Obrador propuso al Gobierno de #NuevoLeon acordar con empresarios que produzcan menos, usen menos #agua, y que así vaya más líquido a la gente. https://t.co/9kW1cPZBmj"
4,27380,0.116,El_Universal_Mx,"#LaMañaneraDeAMLO López Obrador a los oaxaqueños: “Seguimos atendiendo este problema originado por el huracán, vamos a seguir apoyando y ese es el propósito de la visita que va a llevar a cabo el secretario Sandoval para estar con ustedes con todo el apoyo del gobierno federal” https://t.co/kX0AbGi96u"
5,19047,0.119,SinEmbargoMX,"El Secretario de Gobernación, Adán Augusto López, informó que el Gobierno federal y las aerolíneas llegaron a acuerdos para que los vuelos de carga doméstica y las operaciones tipo chárter migren “casi de inmediato” al AIFA https://t.co/LOJTQIPLxG https://t.co/41PU4vxjFX"
6,4318,0.119,El_Universal_Mx,"#LaMañaneraDeAMLO “Pues va bien, no ha habido ninguna notificación del gobierno de España”, dice López Obrador sobre el proceso de ratificación de Quirino Ordaz, exgobernador de Sinaloa, como embajador en ese país https://t.co/c0Pl5eHt5d https://t.co/QuyLegNxMZ"
7,3093,0.119,RicardoAlemanMx,"Si tenían duda, el @TEPJF_informa también está sometido por el gobierno de @lopezobrador_ ! Por eso hoy voto contra la decisión del @INEMexico ! Tienen razón Vargas Llosa y Ernesto Zedillo; con López, México vive la destrucción de su democracia! Al tiempo!"


  0.042159 seconds (119.95 k allocations: 2.745 MiB)


1,19869,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/jbAcSdIuPI"
2,10449,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/BNAdU3bp0n"
3,14765,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa80Meses https://t.co/cR2d88sSeF"
4,10549,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa88meses https://t.co/Yg7Rf8yDoS"
5,26176,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/cxi6tlCTMA"
6,9256,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/jOhSMS35Z5"
7,13983,0.038,epigmenioibarra,"29 JoseE 30 JoseLuisL 31 Jhosivani 32 JulioCesarL El GIEI solicitó investigar el tráfico de drogas escondidas en autobuses como probable motivo del ataque a los normalistas, ya que un quinto autobús que apareció quemado fue omitido en la investigación oficial. #Ayotzinapa81Meses https://t.co/V6gw7dw6ZX"


  0.025442 seconds (119.95 k allocations: 2.745 MiB)


1,6080,0.046,SinEmbargoMX,"#Opinión | Ana Ortega de @THL_Latino ""El consumo de carne o lo que no se habla del calentamiento global"" Si no adoptamos una postura más firme ante el calentamiento global, el panorama de México está a punto de ponerse sombrío https://t.co/HL8v7EBp2x https://t.co/k8Az8hDr5y"
2,639,0.079,bbcmundo,Cambio climático: 5 buenas noticias sobre la lucha contra el calentamiento global (contadas por científicos y expertos) https://t.co/VOKUupHVGN
3,2189,0.088,CNNEE,"▶ Todavía podemos mitigar el daño al planeta por el calentamiento global si exigimos políticas públicas a gobiernos, dicen expertas. https://t.co/7eoc24sboY"
4,27948,0.113,bbcmundo,El calentamiento global avanza y los países parecen incapaces de cumplir con los objetivos de emisiones. https://t.co/xW9mKglT2u
5,13719,0.126,Milenio,▶ ¿Era fuera de lugar? El polémico gol de Aldo Rocha en la final Atlas vs León La anotación se dio por válida y los Zorros empataron el marcador global https://t.co/5QCSfdz0YW https://t.co/omdW15uzOI
6,26946,0.126,CNNEE,"CNN se enorgullece de transmitir en vivo la primera celebración global de Juneteenth desde que se convirtió en un feriado federal en EE.UU. Únete a algunas de las estrellas más famosas mientras juntos alzan sus voces para ""Juneteenth: una celebración mundial por la libertad"". https://t.co/NiFlTlGbTz"
7,12,0.126,CNNEE,"Esta variante de covid-19 detectada por primera vez en la India en febrero se ha vuelto global. Analizamos lo que se sabe hasta ahora y por qué es tan peligrosa. Muy pronto en 'Coronavirus, variante delta: amenaza mundial', un especial presentado por @DonFranciscoTV https://t.co/bqEvLSDWbK"


  0.029511 seconds (119.95 k allocations: 2.745 MiB)


1,24587,0.101,AristeguiOnline,Castillo con coronavirus como corona rompe récord mundial Guinness | #Video https://t.co/SaJIKYierX https://t.co/FmV7YOWKEX
2,22527,0.123,abc_es,"🔴 #ÚLTIMAHORA Don Juan Carlos da negativo en la PCR, tras el positivo en Covid de Rafa Nadal https://t.co/P9Li2qJubQ"
3,20429,0.124,AristeguiOnline,Graves disturbios en Roma durante una manifestación contra el pasaporte Covid https://t.co/CwtlGxCtGs https://t.co/WBz0an5Qc0
4,1801,0.124,NTN24,Cientos de personas protestaron este #27Jul en Roma en contra del certificado sanitario obligatorio en Italia para ingresar a los establecimientos https://t.co/xuE7B2ihz6 https://t.co/tEBwAq7h5h
5,15536,0.13,El_Universal_Mx,"Jorge Carranza dijo estar conforme con la vacuna Sinovac, pues se debe respetar el biológico que le corresponde a cada alcaldía #Video: Salvador Corona https://t.co/OOSopRhLnt"
6,7984,0.131,Milenio,"¿Se contagió a propósito? Marco Melandri, ex piloto italiano, reveló que buscó enfermarse de covid https://t.co/ebB90aGcfP https://t.co/aHCVDb6E0S"
7,16604,0.132,EFEnoticias,"Carlo Ancelotti, técnico italiano del Real Madrid, sigue dando positivo en coronavirus y no integra la expedición a Londres. #UCL https://t.co/m8dp8nJVVu"


In [15]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.092270 seconds (119.95 k allocations: 2.745 MiB)


1,2448,0.0,el_pais,"RT @Planeta_Futuro: Opinión | Parece más fácil devolver a su estado inicial de riqueza a una región que ha sido desarrollada y que, por gue…"
2,28531,0.069,epigmenioibarra,"Indecentes y suicidas me parecen aquellos que, apegados al dogma neoliberal y conservador, pontifican y se oponen a los programas sociales. ¿Qué quieren, perpetuar en la condición de “condenados de la tierra” a quienes han sido históricamente marginados? https://t.co/BfQhgG9BmC"
3,7070,0.069,el_pais,"—¿Cuál ha sido el desahucio más doloroso? —Paco, de 85 años, era un vecino de Lavapiés que llevaba ahí toda la vida, con una pensión de 392 euros, un cáncer terminal y de repente le iban a echar de casa. Acabó siendo muy bonito porque lo paramos https://t.co/hj0nSOH6nT"
4,10072,0.07,el_pais,Eli se recuerda con 21 años rodando camino a una de las zonas más empobrecidas y violentas de la ciudad. Buscaba al artesano gruñón que le enseño el oficio. Se ha convertido en algo único en México: una mujer que hace bicicletas. No ha sido un camino fácil https://t.co/MueHoIwj6M
5,25867,0.071,azucenau,"Sobre la emboscada en Nuevo León donde mataron a 6 elementos de Fuerza Civil el presidente informa que hay apoyo federal para buscar a los responsables de un grupo ya identificado Sobre la crisis del agua,dice, que ya se ha transferido buena cantidad de recursos al estado"
6,1071,0.071,Milenio,"#PolíticaCero | Con la llegada de la satrapía macuspánica hemos visto a los sectores más conservadores, panistas y priistas unirse para salvar al país del comunismo cubanizante y venezolanizado 🎙 Lee y escucha la columna de @jairocalixto https://t.co/pw8Vi5lkmp https://t.co/05jdWFCm9L"
7,17865,0.071,Reforma,"México es el único país del #G20 que no ha anunciado planes para alcanzar las cero emisiones netas para 2050, alertó @WRIMexico. Checa cómo influye esta meta en combatir la #CrisisClimática. https://t.co/5l5vODPHBC"


  0.109603 seconds (119.95 k allocations: 2.745 MiB)


1,2426,0.0,SinEmbargoMX,"Tras concretar la caída de Afganistán en manos de los insurgentes, corren rumores de una nueva era para este país, en el que ya habrían iniciado los asesinatos por venganza y la radio local fue rebautizada como “Voz de la Sharía” https://t.co/ESTiD1r8ke https://t.co/tw1Mdxwk87"
2,25734,0.035,el_pais,La UEFA ordena el cierre parcial del Metropolitano por la exhibición de una bandera con simbología nazi en el estadio del City. Comunica al Atlético que debe dejar vació un espacio de 5.000 personas cuando ya tenía todas las localidades vendidas https://t.co/sNi8zY7JCT
3,14777,0.036,CNNEE,📢 TWITTER SPACE | Gabriel Herrera es venezolano y viaja por el mundo grabando videos para su canal de YouTube. Estaba trabajando en Kyiv cuando comenzaron los ataques rusas y logró salir del país en auto con su pareja y un amigo hacia Rumania. https://t.co/4y3q87Oa6Y
4,4244,0.036,TelemundoNews,"Velan los restos de Debanhi Escobar, la joven de 18 años que desapareció en Nuevo León, México, en medio de teorías que sugieren que fue abusada sexualmente antes de morir. Este es el resumen de lo que ha sucedido hasta el momento 👇🏻 https://t.co/WSXil3ABsq"
5,20019,0.036,teleSURtv,"En #Bolivia 🇧🇴 un grupo de personas lanzó la #LíneaLila, un servicio de transporte operativo exclusivamente por mujeres. Si quieres conocer la historia detrás de esta iniciativa original, mira este video 🎥 . https://t.co/4iyYMVimQV"
6,18279,0.036,EFEnoticias,"La tensión creció en Perú ante la definición de las presidenciales, con Pedro Castillo, quien según sus cuentas ya es el ganador, y con Keiko Fujimori enfocada en intentar demostrar un supuesto ""fraude sistemático"" y anular así miles de votos de su rival. https://t.co/8WmG191btM"
7,13263,0.036,SinEmbargoMX,"Morena arrasa en 4 años | El partido del Presidente logró consolidarse como la principal fuerza política de México y luego de la jornada electoral de este domingo quedó con el poder de 20 entidades, seguido del PAN que tiene bajo su control cinco estados https://t.co/61hfaW1kCJ https://t.co/ZhoVC6EMOA"


  0.080658 seconds (119.95 k allocations: 2.745 MiB)


1,28546,0.0,epigmenioibarra,RT @epigmenioibarra: ¿A qué hora va a empezar a cantar Genaro García Luna? https://t.co/Z5aKeaciar
2,20380,0.08,EFEnoticias,"El líder del PP, Pablo Casado, asegura que la guerra abierta en el PP se puede zanjar hoy mismo con una condición: que Isabel Díaz Ayuso explique en concepto de qué su hermano cobró alrededor de 300.000 euros por un contrato otorgado por su Gobierno. https://t.co/FVI8zPYKHo"
3,8029,0.081,SinEmbargoMX,"@alvaro_delgado @paezvarela @DanyBarraganG @RomiinaGandara @MartinFazMora @INEMexico 🔴 #EnVivo | Martin Faz Mora (@MartinFazMora) Consejero Electoral en @INEMexico ""El elector encontrará los logotipos de todos los partidos políticos, debe tener mucha claridad de quien forma alguna coalición...hay 18 formas de votar"" #LosPeriodistas https://t.co/nqaVgV1vh5 https://t.co/ro1wDYfk0z"
4,27462,0.085,el_pais,"OPINIÓN | ""Mark Zuckerberg ha hecho la cosa más Zuckerberg que se podía esperar: anunciar la pretensión de que vivamos inmersos en un universo paralelo que él pueda controlar"". Por Noelia Ramírez https://t.co/I9OscYovmj"
5,19102,0.085,CNNEE,"Cuando Messi salga al campo en la final de la Copa América el sábado, podría ser su última oportunidad de ganar un trofeo con Argentina. ¿Crees que lo logrará? https://t.co/ZUKgctgjRm"
6,29139,0.085,el_pais,"""Estamos en Europa y vivimos muy tranquilos, pero esto puede acabar de la noche a la mañana"", cuenta Roberto, colombiano que vive desde hace solo tres meses en Barcelona. Es una de las personas que ha acudido al Consulado de Ucrania para alistarse https://t.co/F3hvD5bVXE"
7,22824,0.085,CNNEE,🍕 Malas noticias para los amantes de la pizza: la clásica Hot-N-Ready de US$ 5 de Little Caesars ahora cuesta un 11% más. Es su primer aumento de precio en casi 25 años. https://t.co/6zcoOzc1qw


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^\textsf{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Defina $H^\textsf{TF}_d$, donde $TF$ es la frecuencia de término normalizada.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
